In [ ]:
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Preparação dos dados

### Coleta de dados

In [ ]:
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)
worksheet = gc.open('pizzaria').sheet1

In [ ]:
# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)
# Convert to a DataFrame and render.
import pandas as pd
df=pd.DataFrame(rows[1:],columns=rows[0])

[['conjunto', 'saudacoes', 'ver_cardapio', 'fazer_pedido', 'para_entrega', 'para_retirar', 'endereco_entrega', 'forma_pagamento'], ['Treino', 'Oi', 'Gostaria de ver o cardápio, por favor', 'oie, eu queria pedir uma pizza de Calabresa', 'para entregar aqui em casa', 'vo busca', 'É para entregar Av Afonso Pena 356', 'Dinheiro'], ['Treino', 'oi', 'Ola, eu gostaria de fazer um pedido quais as opções de pizza?', 'Me vê uma pizza de bacon, outra de portuguesa e uma coca-cola', 'pode entrega', 'vou buscar', 'A entrega é para a Rua da Paz 1001', 'no dinheiro'], ['Treino', 'Ola', 'Olá, poderia me mostrar o cardapio?', 'Gostaria de uma de Portuguesa e uma Fragon com catupiry por gentileza', 'vou mandar o endereço', 'retirada no balcão', 'Rua UFMS, Facom', 'em dinheiro'], ['Treino', 'Olá', 'Poderia mandar o cardápio?', 'Gostaria de uma pizza de Bacon e outra de Calabresa', 'manda entregar', 'tô passando ai mais tarde', 'Rua Afonso Pena, 3002.', 'no cartão'], ['Treino', 'Oie', 'favor o cardápio', 

In [ ]:
intencoes = ['saudacoes', 'ver_cardapio', 'fazer_pedido', 'para_entrega', 'para_retirar', 'endereco_entrega', 'forma_pagamento']
xtrain_global = []
ytrain_global = []
for intencao in intencoes:
    lintencao = df[df['conjunto']=='Treino'][intencao].values.tolist()
    xtrain_global += lintencao
    ytrain_global += [intencao]*len(lintencao)

In [ ]:
xtest = []
ytest = []
for intencao in intencoes:
    lintencao = df[df['conjunto']=='Teste'][intencao].values.tolist()
    xtest += lintencao
    ytest += [intencao]*len(lintencao)

In [ ]:
len(xtest),len(xtrain_global)

(70, 357)

In [ ]:
xtest,ytest

## split de treino em validação e treino

In [ ]:
ytrain_global[200],xtrain_global[200]

('para_entrega', 'aqui no meu apartamento')

In [ ]:
import sklearn.model_selection as model_selection

In [ ]:
xtrain,xval,ytrain,yval = model_selection.train_test_split(xtrain_global,ytrain_global,test_size=0.25,stratify=ytrain_global)

In [ ]:
np.unique(ytrain,return_counts=True)

(array(['endereco_entrega', 'fazer_pedido', 'forma_pagamento',
        'para_entrega', 'para_retirar', 'saudacoes', 'ver_cardapio'],
       dtype='<U16'), array([38, 39, 38, 38, 38, 38, 38]))

## Pré-processamento

O pré-processamento consiste em transformar todas as sentenças em vetores utilizando sentence-embeddings 

In [ ]:
!pip install -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
converter = SentenceTransformer('multi-qa-distilbert-cos-v1')

In [ ]:
sent1=converter.encode('manda um pizza de calabresa')

In [ ]:
sent1.shape

(768,)

In [ ]:
sentences = converter.encode(['vou querer 3 pizzas de calabresa','vou querer 3 calabresa'])

In [ ]:
for sent in sentences:
    print(np.linalg.norm(sent1-sent))

0.69445914
0.9674724


In [ ]:
xtrain[:20]

['rua ojogo, 092',
 'vo pega aí',
 'meu pai vai pegar aí',
 'faz a entrega por favor',
 'vou mandar o endereço',
 'rua taacabando, 378',
 'para entregar aqui em casa',
 'pode me mandar foto do que tem no cardápio?',
 'entrega entrega',
 'tô passando ai mais tarde',
 'entrega',
 'Ola',
 'pode mandar lá pro meu apartamento',
 'buscar, por favor',
 'pegar',
 'adoraria uma pizza de bacon e uma coca cola',
 'vou mandar o local',
 'entrega',
 'no cartão',
 'manda pra cá']

In [ ]:
xval[:30]

['quero uma pizza de bacon e uma  de calabresa',
 'Rua ufms 666',
 'pode entregar',
 'tenho que ir aí mesmo',
 'entrega',
 'Boa tarde chefe',
 'buscar ok',
 'rua acabousera, 389',
 'cartão',
 'pra entrega la na minha mãe',
 'Bom dia',
 'ai bicho, quero ver o menu de hoje',
 'manda pra mim',
 'Ola, eu gostaria de fazer um pedido quais as opções de pizza?',
 'rua marechal rondon 738',
 'aopa, bao? me da um cardápio ai',
 'já pego',
 'Me vê uma pizza de bacon, outra de portuguesa e uma coca-cola',
 'rua terenos 5489',
 'vou pedir uma pizza de calabresa',
 'to indo buscar',
 'tenho que pegar aí',
 'no cartão de crédito',
 'Rua Rosa Azul, 10',
 'rua perdiojogo, 999',
 'Av Afonso Pena 350',
 'com dinheiro',
 'Ola',
 'cara, me ve uma pizza de frango com catupiry, na moral',
 'eu passo ai']

In [ ]:
xtest[:10]

['Oi tudo bem',
 'ola já estão atendendo?',
 'ola já estão fazendo pedido?',
 'Salve meu chefe',
 'Saudacoes',
 'E aí pessoal, tudo bem?',
 'Gostaria de fazer um pedodo',
 'Boa noite',
 'Boa tarde, estão abertos?',
 'Bom dia, gostaria de fazer o pedido']

In [ ]:
xtrain_emb = converter.encode(xtrain)
xval_emb = converter.encode(xval)
xtest_emb = converter.encode(xtest)

In [ ]:
xtrain[1],xtrain_emb[1][:10]

('vo pega aí',
 array([-0.00118349,  0.00042262,  0.00198435,  0.04273261,  0.05693456,
         0.06459454, -0.00399817,  0.06375381, -0.03738355, -0.00389999],
       dtype=float32))

# Selecionar o Modelo

In [ ]:
import sklearn.neighbors as neighbors

In [ ]:
model = neighbors.KNeighborsClassifier(n_neighbors=5)

In [ ]:
model.fit(xtrain_emb,ytrain)

KNeighborsClassifier()

In [ ]:
model.predict([converter.encode('vou querer 3 calabresas por favor')])

array(['para_entrega'], dtype='<U16')

# Ajuste de parâmetros

In [ ]:
import sklearn.metrics as metrics

In [ ]:
for i in range(1,15,2):
    model = neighbors.KNeighborsClassifier(n_neighbors=i,weights="distance")
    model.fit(xtrain_emb,ytrain)
    pred = model.predict(xval_emb)
    print(f"k={i}")
    print(metrics.classification_report(yval,pred))

k=1
                  precision    recall  f1-score   support

endereco_entrega       0.93      1.00      0.96        13
    fazer_pedido       0.92      1.00      0.96        12
 forma_pagamento       1.00      1.00      1.00        13
    para_entrega       1.00      0.85      0.92        13
    para_retirar       0.93      1.00      0.96        13
       saudacoes       0.92      0.85      0.88        13
    ver_cardapio       0.92      0.92      0.92        13

        accuracy                           0.94        90
       macro avg       0.95      0.95      0.94        90
    weighted avg       0.95      0.94      0.94        90

k=3
                  precision    recall  f1-score   support

endereco_entrega       0.93      1.00      0.96        13
    fazer_pedido       0.86      1.00      0.92        12
 forma_pagamento       1.00      0.92      0.96        13
    para_entrega       0.86      0.92      0.89        13
    para_retirar       0.92      0.92      0.92        13
  

# Avaliação do Modelo

In [ ]:
model = neighbors.KNeighborsClassifier(n_neighbors=5,weights="distance")
model.fit(xtrain_emb,ytrain)
pred = model.predict(xtest_emb)
print(f"k={i}")
print(metrics.classification_report(ytest,pred))

k=13
                  precision    recall  f1-score   support

endereco_entrega       0.90      0.90      0.90        10
    fazer_pedido       0.91      1.00      0.95        10
 forma_pagamento       0.90      0.90      0.90        10
    para_entrega       0.90      0.90      0.90        10
    para_retirar       0.82      0.90      0.86        10
       saudacoes       1.00      0.80      0.89        10
    ver_cardapio       0.80      0.80      0.80        10

        accuracy                           0.89        70
       macro avg       0.89      0.89      0.89        70
    weighted avg       0.89      0.89      0.89        70



# Teste

In [ ]:
model.predict([converter.encode('me veja 3 pizzas de calabresa por favor')])

array(['fazer_pedido'], dtype='<U16')

# Modelo final

In [ ]:
model.fit(converter.encode(xtrain_global+xtest),ytrain_global+ytest)

KNeighborsClassifier(weights='distance')

In [ ]:
from joblib import dump, load

In [ ]:
dump(model, 'pizzaria.joblib') 

['pizzaria.joblib']

In [ ]:
modelo_final = load('pizzaria.joblib') 

In [ ]:
modelo_final.predict([converter.encode('vou querer 3 calabresas')])

array(['fazer_pedido'], dtype='<U16')

In [ ]:
def convert_num(n):
    valores = {'um':1,'uma':1,'dois':2,'duas':2,'tres':3,'quatro':4,'cinco':5,'seis':6,'sete':7,'oito':8,'nove':9,'dez':10}
    ret = 0
    if n.isnumeric():
        ret = int(n)
    else:
        if n in valores.keys():
            ret = valores[n]
    return ret

In [ ]:
!pip install unidecode

In [ ]:
from nltk.tokenize import TweetTokenizer
from unidecode import unidecode
tknzr = TweetTokenizer()

In [ ]:
lista_entidades = [
'item:coca,cocas,coca-cola,coquinha,guarana,agua,guaranas',
'item:pizza de calabresa,calabresa,pizza de bacon,bacon,beicon,portuguesa,pizza de frango com catupiry,franco com catupiry,catupiry,catupiri,pizza de presunto,presunto,calabresas,portuguesas',
'num:1,2,3,4,5,6,7,8,9,10',
'num:um,dois,tres,quatro,cinco,seis,sete,oito,nove,dez,uma,duas'
]
entidades = dict()
def load_entidades(lista_entidades):
            for line in lista_entidades:
                entidade,valores = line.split(':')
                str_valores = valores[:]
                valores = str_valores.split(',')
                for valor in valores:
                    if valor not in entidades.keys():
                        entidades[valor] = entidade
load_entidades(lista_entidades)
def find_entidades(texto):
    ret = dict()
    for token in tknzr.tokenize(texto):
        token = token.lower()
        token = unidecode(token)
        if token in entidades.keys():
            ent = entidades[token]
            if ent not in ret.keys():
                ret[ent] = [token]
            else:
                ret[ent] += [token]
    return ret

In [ ]:
def convert_num(n):
    valores = {'um':1,'uma':1,'dois':2,'duas':2,'tres':3,'quatro':4,'cinco':5,'seis':6,'sete':7,'oito':8,'nove':9,'dez':10}
    ret = 0
    if n.isnumeric():
        ret = int(n)
    else:
        if n in valores.keys():
            ret = valores[n]
    return ret

In [ ]:
print(find_entidades('me ve uma pizza de presunto ai'))

{'num': ['uma'], 'item': ['presunto']}


In [ ]:
valor_cardapio = {
    'calabresa':28.0,
    'bacon':30.0,
    'portuguesa':32.0,
    'catupiry':32.0,
    'presunto':29.0,
    'coca':10.0,
    'guarana':10.0
}

In [ ]:
def str_menu(h):
    rstr = ''
    for item in h:
        rstr += f"{item:<10}  {h[item]:>5}\n"
    return rstr

In [ ]:
print(str_menu(valor_cardapio))

calabresa    28.0
bacon        30.0
portuguesa   32.0
catupiry     32.0
presunto     29.0
coca         10.0
guarana      10.0



# **Construção do ChatBot**

Instalando e importando as bibliotecas necessaria para a integração com o bot, após rodar a primeira vez, reinicie o ambiente de execução

In [ ]:
!pip install python-telegram-bot --upgrade

In [ ]:
from telegram import Update, ForceReply
from telegram.ext import Updater, MessageHandler, Filters, CallbackContext

Criando verificação sobre a palavra pix para poder fazer a checagem no método de mensagem

In [ ]:
worksheet2 = gc.open('pix').sheet1
rows2 = worksheet2.get_all_values()
print(rows2)
df2=pd.DataFrame(rows2[1:],columns=rows2[0])
##
intencoes2 = ['pix','nao_pix']
xtrain_global2 = []
ytrain_global2 = []
for intencao in intencoes2:
    lintencao2 = df2[df2['conjunto']=='Treino'][intencao].values.tolist()
    xtrain_global2 += lintencao2
    ytrain_global2 += [intencao]*len(lintencao2)
xtest2 = []
ytest2 = []
for intencao in intencoes2:
    lintencao2 = df2[df2['conjunto']=='Teste'][intencao].values.tolist()
    xtest2 += lintencao2
    ytest2 += [intencao]*len(lintencao2)
xtrain2,xval2,ytrain2,yval2 = model_selection.train_test_split(xtrain_global2,ytrain_global2,test_size=0.25,stratify=ytrain_global2)
np.unique(ytrain2,return_counts=True)
converter2 = SentenceTransformer('multi-qa-distilbert-cos-v1')
xtrain_emb2 = converter2.encode(xtrain2)
xval_emb2 = converter2.encode(xval2)
xtest_emb2 = converter2.encode(xtest2)
model2 = neighbors.KNeighborsClassifier(n_neighbors=5)
model2.fit(xtrain_emb2,ytrain2)

[['pix', 'conjunto', 'nao_pix'], ['pix', 'Treino', 'dinheiro'], ['pox', 'Treino', 'cartao'], ['pex', 'Treino', 'debito'], ['Pix', 'Treino', 'crédito'], ['pix', 'Treino', 'dinheiro'], ['pix', 'Treino', 'dinheiro'], ['pox', 'Treino', 'cartao'], ['pex', 'Treino', 'debito'], ['Pix', 'Treino', 'crédito'], ['pix', 'Treino', 'crédito'], ['pix', 'Treino', 'dinheiro'], ['pox', 'Treino', 'cartao'], ['pex', 'Treino', 'debito'], ['Pix', 'Treino', 'crédito'], ['pix', 'Treino', 'crédito'], ['pix', 'Treino', 'dinheiro'], ['pox', 'Treino', 'cartao'], ['pex', 'Treino', 'debito'], ['Pix', 'Treino', 'crédito'], ['pix', 'Treino', 'debito'], ['pix', 'Treino', 'dinheiro'], ['pox', 'Treino', 'cartao'], ['pix', 'Teste', 'dinheiro'], ['pix', 'Teste', 'cartao'], ['Pix', 'Teste', 'debito'], ['Pex', 'Teste', 'crédito'], ['Pox', 'Teste', 'dinheiro']]


KNeighborsClassifier()

In [ ]:
model2.predict([converter2.encode('px')])

array(['pix'], dtype='<U7')

In [ ]:
dump(model2, 'pix.joblib') 
modelo_final2 = load('pix.joblib') 
modelo_final2.predict([converter.encode('debisto')])

array(['nao_pix'], dtype='<U7')

Criando método que retornará as respostas do bot

In [ ]:
from datetime import datetime
totalFinal = 0 #total com frete
total = 0
def getMessage(cliente):
  global total
  global totalFinal
  #intencoes = ['saudacoes', 'ver_cardapio', 'fazer_pedido', 'para_entrega', 'para_retirar', 'endereco_entrega', 'forma_pagamento']
  recomendacao = ['Calabresa','Presunto','Bacon','Portuguesa','Frango Catupiry','Portuguesa','Frango Catupiry']
  pedidos = []
  msg = ''
  pred = modelo_final.predict([converter.encode(cliente)])[0]
  predFormaPagamento = modelo_final2.predict([converter.encode(cliente)])[0]
  print(pred)
  if pred == "saudacoes":
    msg = 'Boa noite, a recomentodação do chefe hoje é :'+recomendacao[datetime.today().weekday()]
  elif pred == 'ver_cardapio':
    msg = str_menu(valor_cardapio)
  elif pred == 'fazer_pedido':
        ent = find_entidades(cliente)
        print(ent)
        try:
            joined = [[x,y] for x,y in zip(ent['num'],ent['item'])]
            pedidos.append(joined)
            for joined in pedidos:
              for n,item in joined:
                nvalor = convert_num(n)
                if item in valor_cardapio.keys():
                  vitem = valor_cardapio[item]
                  total += vitem*nvalor
                  msg += '%s %s %04.2f\n'%(n,item,vitem*nvalor)
            totalFinal = total + 7.0;
            msg += ('Ok, tudo ficou R$ %4.2f, será para entrega ou retirar?\n'%total)
        except: 
            msg += 'Não te entendi direito'
  elif pred == 'para_entrega':
        msg += ('Com entrega ficou R$ %4.2f, qual o endereço de entrega?'%totalFinal)
  elif pred == 'para_retirar':      
        msg += 'Qual a forma de pagamento?'      
  elif pred == 'endereco_entrega':      
        msg += 'Qual a forma de pagamento?'
  elif pred == 'forma_pagamento':
        if predFormaPagamento == 'pix':
            msg += 'Segue o número do pix: 231564789476315. Seu pedido sai em no máximo 50 minutos, obrigado pela preferência'
        else:
            msg += 'Seu pedido sai em no máximo 50 minutos, obrigado pela preferência'
        total = 0
        totalFinal = 0          
  else:
        msg = 'Não, entendi. \n'+str_menu(valor_cardapio)
  print(msg)
  return msg

Definição do método que ecoará sempre que mandada uma mensagem ao bot

In [1]:
def echo(update: Update, context: CallbackContext) -> None:
    update.message.reply_text(getMessage(update.message.text))

NameError: ignored

Método main que contem a declaração do bot, handler e inicialização do bot

In [ ]:
def main() -> None:
    """Start the bot."""
    updater = Updater("5109675251:AAFjnQx_NLQbt67w5nSeZMsh8fzHcyNVcA8")
    # Para obter a chave converse com o BotFather no telegram (O bot cria contas de bots dentro do telegram)
    dispatcher = updater.dispatcher
    # on non command i.e message - echo the message on Telegram
    dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, echo))
    # Start the Bot
    updater.start_polling()
    # Leave the bot waiting for messages
    updater.idle()

In [ ]:
main()

saudacoes
Boa noite, a recomentodação do chefe hoje é :Frango Catupiry
para_retirar
Qual a forma de pagamento?
fazer_pedido
{'num': ['uma'], 'item': ['catupiri']}
Ok, tudo ficou R$ 0.00, será para entrega ou retirar?

fazer_pedido
{'num': ['uma'], 'item': ['calabresa']}
uma calabresa 28.00
Ok, tudo ficou R$ 28.00, será para entrega ou retirar?

fazer_pedido
{'num': ['uma'], 'item': ['catupiry']}
uma catupiry 32.00
Ok, tudo ficou R$ 60.00, será para entrega ou retirar?

para_retirar
Qual a forma de pagamento?
forma_pagamento
Seu pedido chega em no máximo 50 minutos, obrigado pela preferência
